In [1]:
import os
import sys
import ast
import json
import asyncio
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Image, display
from tqdm.asyncio import tqdm_asyncio

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from config import Config
from services.openai_service import OpenAIService
from services.tavily_service import TavilyService
from services.feature_extractor_agent_v2 import FeatureExtractor

# Load configuration
config = Config()
openai_service = OpenAIService(config.OPENAI_API_KEY, config)
tavily_service = TavilyService(config.TAVILY_API_KEY)
feature_extraction = FeatureExtractor(openai_service, tavily_service, max_attempts=5)

# Set up Pandas and Seaborn
sns.set()
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
data = pd.read_csv("../data/extracted_data_combined.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1105 entries, 0 to 1104
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1105 non-null   object
 1   data    1105 non-null   object
dtypes: object(2)
memory usage: 17.4+ KB


In [3]:
data.head()

,id,data
0,Z9-PC-DEVKIT,"{'name': 'ZumLink Series OEM ZPC ZPCSR', 'manufacturer': 'FreeWave Technologies', 'form_factor': 'ZPC: 63.5 x 38.1 x 10.2 mm, ZPCSR: 76.2 x 50.8 x 10.2 mm', 'processor': 'ARM Cortex-A8 1 GHz', 'core_count': 'Not available', 'processor_tdp': 'Not available', 'memory': 'RAM: 512 MB, Storage: 4 GB', 'io': 'Dual Row 20-pin header, Ethernet, Power, Serial, USB (Micro USB), RF Connector (MMCX)', 'operating_system': 'Debian-based Linux', 'environmentals': 'Operating Temperature: -40°C to 75°C (-40°F to 167°F), Humidity: 95% non-condensing', 'certifications': 'Class I Division 2 certified, UL, RoHS Directive 2011/65/EU', 'short_summary': 'The ZumLink Series OEM ZPC ZPCSR is an industrial Ethernet radio designed for secure, long-range, low-power wireless communications in rugged environments.', 'full_summary': 'The ZumLink Series OEM ZPC ZPCSR by FreeWave Technologies is an industrial Ethernet radio designed for secure data collection, transport, and control in rugged environments. It operates in the unlicensed 900 MHz spectrum using Frequency Hopping Spread Spectrum (FHSS) technology, providing long-range, low-power wireless communications with enhanced network efficiency and reliability.', 'full_product_description': ""Designed and manufactured by FreeWave Technologies, the ZumLink Series OEM ZPC ZPCSR is an industrial Ethernet radio that operates in the unlicensed 900 MHz spectrum using Frequency Hopping Spread Spectrum (FHSS) technology. It offers secure data transfer with RF link rates up to 4 Mbps and features such as packet compression, packet aggregation, and forward error correction to maximize network throughput, efficiency, and reliability. The radios are ideal for integration into networks, systems, and devices used in industries like oil and gas, precision agriculture, water/wastewater, smart cities, and utilities. They provide a Cost, Size, Weight, and Power (CSWaP) advantage with a small, lightweight form factor and low power consumption. The ZumLink Series is also software upgradable to include FreeWave's IQ Application Environment, a Linux-based operating system for developing and deploying third-party applications. The product is certified for safe use in hazardous locations and complies with UL and RoHS standards.""}"
1,SDK-AM3517-10-256512R,"{'name': 'Zoom AM Experimenter Kit', 'manufacturer': 'Logic Product Development', 'form_factor': 'Not available', 'processor': 'Texas Instruments AM Processor', 'core_count': 'Not available', 'processor_tdp': 'Not available', 'memory': 'Not available', 'io': 'SDMMC card slot, HDMI video port, USB Type B port, S-Video port, RJ Ethernet jack, USB On-the-Go mini-AB port, USB host Type A port, DB-9 serial debug port', 'operating_system': 'Linux', 'environmentals': 'Not available', 'certifications': 'Not available', 'short_summary': 'The Zoom AM Experimenter Kit is a development kit for evaluating the functionality of the Texas Instruments AM processor.', 'full_summary': 'The Zoom AM Experimenter Kit provides a product-ready hardware and software platform for evaluating the functionality of the Texas Instruments AM processor. It includes the AM SOMM, experimenter baseboard, various cables, and power adapters.', 'full_product_description': 'The Zoom AM Experimenter Kit is designed for engineering development, demonstration, or evaluation purposes. It includes the AM SOMM, experimenter baseboard, null-modem serial cable, USB A to mini-B cable, and a regulated 5V power supply with adapters for different regions. The kit supports terminal emulation programs and comes with pre-installed U-Boot bootloader and Linux kernel. Additional documentation and software resources are available on the Logic Product Development website.'}"
2,LPC3180-DEV-KIT,"{'name': 'LPC Development Kit', 'manufacturer': 'NXP Semiconductors', 'form_factor': 'Prototyping form-factor', 'processor': 'NXP LPC', 'core_count': '1', 'processor_tdp': 'Not available', 'memory': '32 MB SDRAM, 64 MB NAN

In [4]:
data["data"] = data["data"].apply(ast.literal_eval)
data_expanded = pd.json_normalize(data["data"])
df_expanded = pd.concat([data["id"], data_expanded], axis=1)
df_expanded.head()

,id,name,manufacturer,form_factor,processor,core_count,processor_tdp,memory,io,operating_system,environmentals,certifications,short_summary,full_summary,full_product_description
0,Z9-PC-DEVKIT,ZumLink Series OEM ZPC ZPCSR,FreeWave Technologies,"ZPC: 63.5 x 38.1 x 10.2 mm, ZPCSR: 76.2 x 50.8 x 10.2 mm",ARM Cortex-A8 1 GHz,Not available,Not available,"RAM: 512 MB, Storage: 4 GB","Dual Row 20-pin header, Ethernet, Power, Serial, USB (Micro USB), RF Connector (MMCX)",Debian-based Linux,"Operating Temperature: -40°C to 75°C (-40°F to 167°F), Humidity: 95% non-condensing","Class I Division 2 certified, UL, RoHS Directive 2011/65/EU","The ZumLink Series OEM ZPC ZPCSR is an industrial Ethernet radio designed for secure, long-range, low-power wireless communications in rugged environments.","The ZumLink Series OEM ZPC ZPCSR by FreeWave Technologies is an industrial Ethernet radio designed for secure data collection, transport, and control in rugged environments. It operates in the unlicensed 900 MHz spectrum using Frequency Hopping Spread Spectrum (FHSS) technology, providing long-range, low-power wireless communications with enhanced network efficiency and reliability.","Designed and manufactured by FreeWave Technologies, the ZumLink Series OEM ZPC ZPCSR is an industrial Ethernet radio that operates in the unlicensed 900 MHz spectrum using Frequency Hopping Spread Spectrum (FHSS) technology. It offers secure data transfer with RF link rates up to 4 Mbps and features such as packet compression, packet aggregation, and forward error correction to maximize network throughput, efficiency, and reliability. The radios are ideal for integration into networks, systems, and devices used in industries like oil and gas, precision agriculture, water/wastewater, smart cities, and utilities. They provide a Cost, Size, Weight, and Power (CSWaP) advantage with a small, lightweight form factor and low power consumption. The ZumLink Series is also software upgradable to include FreeWave's IQ Application Environment, a Linux-based operating system for developing and deploying third-party applications. The product is certified for safe use in hazardous locations and complies with UL and RoHS standards."
1,SDK-AM3517-10-256512R,Zoom AM Experimenter Kit,Logic Product Development,Not available,Texas Instruments AM Processor,Not available,Not available,Not available,"SDMMC card slot, HDMI video port, USB Type B port, S-Video port, RJ Ethernet jack, USB On-the-Go mini-AB port, USB host Type A port, DB-9 serial debug port",Linux,Not available,Not available,The Zoom AM Experimenter Kit is a development kit for evaluating the functionality of the Texas Instruments AM processor.,"The Zoom AM Experimenter Kit provides a product-ready hardware and software platform for evaluating the functionality of the Texas Instruments AM processor. It includes the AM SOMM, experimenter baseboard, various cables, and power adapters.","The Zoom AM Experimenter Kit is designed for engineering development, demonstration, or evaluation purposes. It includes the AM SOMM, experimenter baseboard, null-modem serial cable, USB A to mini-B cable, and a regulated 5V power supply with adapters for different regions. The kit supports terminal emulation programs and comes with pre-installed U-Boot bootloader and Linux kernel. Additional documentation and software resources are available on the Logic Product Development website."
2,LPC3180-DEV-KIT,LPC Development Kit,NXP Semiconductors,Prototyping form-factor,NXP LPC,1,Not available,"32 MB SDRAM, 64 MB NAND Flash","UARTs, I2C, SPI, 10-bit ADC, GPIO, USB OTG, RS-232, Ethernet, SD Card Slot, LCD Keypad",Embedded Linux,Not available,Not available,The LPC Development Kit provides a stable platform for building powerful user applications within an ARM Linux environment.,The LPC Development Kit provides a stable platform for building powerful user applications within an ARM Linux environment. It supports popular mobile technologies and comes preload

In [5]:
df_expanded = df_expanded.replace("Not available", np.nan)
df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1105 entries, 0 to 1104
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        1105 non-null   object
 1   name                      1105 non-null   object
 2   manufacturer              1103 non-null   object
 3   form_factor               1088 non-null   object
 4   processor                 1054 non-null   object
 5   core_count                861 non-null    object
 6   processor_tdp             521 non-null    object
 7   memory                    1039 non-null   object
 8   io                        1104 non-null   object
 9   operating_system          811 non-null    object
 10  environmentals            998 non-null    object
 11  certifications            682 non-null    object
 12  short_summary             1105 non-null   object
 13  full_summary              1105 non-null   object
 14  full_product_description

## Duplicate names

In [6]:
duplicate_products = df_expanded[df_expanded.duplicated(subset=["name"], keep=False)]
duplicate_products = duplicate_products.sort_values(by="name")

duplicate_products.info()

for name, group in duplicate_products.groupby("name"):
    print(f"Name: {name}")
    for index, row in group.iterrows():
        print(
            f"Name: {row['name']}, Manufacturer: {row['manufacturer']}, Processor: {row['processor']}, Memory: {row['memory']}, Summary: {row['short_summary']}"
        )

<class 'pandas.core.frame.DataFrame'>
Index: 375 entries, 199 to 982
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        375 non-null    object
 1   name                      375 non-null    object
 2   manufacturer              373 non-null    object
 3   form_factor               374 non-null    object
 4   processor                 364 non-null    object
 5   core_count                298 non-null    object
 6   processor_tdp             237 non-null    object
 7   memory                    365 non-null    object
 8   io                        375 non-null    object
 9   operating_system          266 non-null    object
 10  environmentals            366 non-null    object
 11  certifications            237 non-null    object
 12  short_summary             375 non-null    object
 13  full_summary              375 non-null    object
 14  full_product_description  375

In [7]:
df_expanded["null_count"] = df_expanded.isnull().sum(axis=1)
df_expanded = df_expanded.sort_values(["null_count", "name"], ascending=[True, True])
# Group by 'name' and aggregate the 'id' column into a list to capture all ids for duplicates
df_grouped = (
    df_expanded.groupby("name")
    .agg(
        {
            "id": lambda x: ",".join(x),
            "null_count": "first",
            **{col: "first" for col in df_expanded.columns if col not in ["id", "null_count", "name"]},
        }
    )
    .reset_index()
)

# Rename the 'id' column to 'ids'
df_grouped.rename(columns={"id": "ids"}, inplace=True)

# Drop the 'null_count' column as it's no longer needed
df_grouped = df_grouped.drop(columns=["null_count"])

df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808 entries, 0 to 807
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   name                      808 non-null    object
 1   ids                       808 non-null    object
 2   manufacturer              808 non-null    object
 3   form_factor               792 non-null    object
 4   processor                 765 non-null    object
 5   core_count                635 non-null    object
 6   processor_tdp             340 non-null    object
 7   memory                    750 non-null    object
 8   io                        807 non-null    object
 9   operating_system          613 non-null    object
 10  environmentals            707 non-null    object
 11  certifications            507 non-null    object
 12  short_summary             808 non-null    object
 13  full_summary              808 non-null    object
 14  full_product_description  

In [8]:
df_grouped.head()

,name,ids,manufacturer,form_factor,processor,core_count,processor_tdp,memory,io,operating_system,environmentals,certifications,short_summary,full_summary,full_product_description
0,A Quad Core Tablet Processor,A33-OLinuXino-4GB,Allwinner Technology,"TFBGA 324 balls, 0.65mm ball pitch, 14x14 mm",ARM Cortex-A7,4,None,Supports up to 4GB DDR3/DDR3L SDRAM,"USB, SD/MMC, UART, SPI, TWI, GPIO, CSI, DSI, LVDS, Audio Codec",Android 5.0 Lollipop,Operating Temperature: -40°C to 85°C,None,"The Allwinner A is a power-efficient quad-core tablet processor based on ARM Cortex-A7 CPU architecture with Mali-400 MP2 GPU, offering high system integration and low power consumption.","The Allwinner A is a remarkably power-efficient quad-core tablet processor based on ARM Cortex-A7 CPU architecture along with Mali-400 MP2 GPU. It is highly competitive in terms of system cost due to its high system integration, capable of delivering excellent user experience while maintaining ultra-low power consumption. The processor supports high-definition 1080p video processing, various mainstream video standards, and a wide range of integrated I/Os.","The Allwinner A is a power-efficient quad-core tablet processor based on ARM Cortex-A7 CPU architecture and Mali-400 MP2 GPU. It offers high system integration, capable of delivering excellent user experience with ultra-low power consumption. Key features include support for high-definition 1080p video processing, various mainstream video standards (H.264, VP8, MPEG-1/2/4, JPEG/MJPEG), and a wide range of integrated I/Os such as USB, SD/MMC, UART, SPI, TWI, GPIO, CSI, DSI, LVDS, and Audio Codec. The processor also supports advanced features like hardware virtualization, LPAE, and DVFS for independent power domain management."
1,AE Mini PC,114110148,Aetina,Overall Size 100 mm x 87 mm,NVIDIA Jetson Xavier NX,6,15W,8 GB 128-bit LPDDR4x 1600 MHz 51.2 GB/s,"HDMI, USB Type-A, USB Type-C, Gigabit Ethernet, TF Card slot, RS232, RS485, CAN, SPI",Ubuntu with NVIDIA L4T,"Operating Temperature -25°C to 80°C, Storage Temperature -40°C to 85°C, Storage Humidity 95% non-condensing",NVIDIA-Certified Systems,"AE Mini PC is an embedded artificial intelligence computer with NVIDIA Jetson Xavier NX, offering high performance and rich interfaces.","AE Mini PC is an embedded artificial intelligence computer capable of empowering many end devices with 21 TOPS computing power. It features a fast, passive thermal design that meets industrial standards, including anti-vibration and anti-static properties. The AE Mini PC is equipped with the NVIDIA Jetson Xavier NX processor, 8 GB LPDDR4x memory, and offers a variety of interfaces such as HDMI, USB, Ethernet, and more.","The AE Mini PC is an embedded artificial intelligence computer designed to empower a wide range of end devices. It features the NVIDIA Jetson Xavier NX processor, providing 21 TOPS of AI performance. The device includes a passive thermal design that meets industrial standards for anti-vibration and anti-static properties. It comes with 8 GB of 128-bit LPDDR4x memory, 16 GB eMMC storage, and a variety of interfaces including HDMI, USB Type-A, USB Type-C, Gigabit Ethernet, TF Card slot, RS232, RS485, CAN, and SPI. The AE Mini PC runs on Ubuntu with NVIDIA L4T, and supports a wide range of AI and multimedia applications through the NVIDIA JetPack SDK."
2,AGS High Density GPU Server,AGS-923I-R18A1E,Advantech,2U,Intel Xeon E5-2600 v3,6 to 18 cores,85W to 145W,"DDR4 2133 MHz, up to 512GB","4x Gigabit Ethernet, 2x USB 3.0, 2x USB 2.0, 1x VGA, 1x COM","Windows, Linux","Operating Temperature: 0°C to 40°C, Non-operating Temperature: -40°C to 70°C, Operating Relative Humidity: 10% to 85% non-condensing, Non-operating Relative Humidity: 10% to 95% non-condensing","FCC Class A, RoHS Compliant",The AGS High Density GPU Server by Advantech is a 2U server designed for industrial applications requiring high computing power and multiple expansion slots. It supports Intel Xeon E5-2600 v3 proces

In [9]:
df_grouped = df_grouped.replace(np.nan, "Not available")
df_grouped.to_csv("../data/extracted_data_grouped.csv", index=False)